## Event reproducibility in ALM
### data from Li et al. 2015
See the paper:    
*"A motor cortex circuit for motor planning and movement"*    
Nuo Li, Tsai-Wen Chen, Zengcai V. Guo, Charles R. Gerfen & Karel Svoboda    
Nature 519, 51–56 (05 March 2015) doi: 10.1038/nature14178

In [1]:
from platform import python_version
print(python_version())

%run -i 'imports_functions.py' 

%matplotlib inline

3.10.4


### Data preprocessing

The original files `alm-2` containing the DF/F traces is available on [CRCNS](http://crcns.org/data-sets/motor-cortex/alm-2).

Here, to speed up the processing and reduce the dependencies, we use the Ca spiketrains extracted using the `deconvolve` function available in the library [OASIS](https://github.com/j-friedrich/OASIS/pull/19) (with all default parameters, and `penalty` of 1).

In [2]:
exp_path = os.getcwd() + "/svoboda/"

frame_duration = 0.1 # sec, 10 frames per second, see Methods in Franco and Goard 2021

We take the separate 57 sessions independently (some are not used because they have no significant events).

In [3]:
ophys_experiments = [
    # animal 19
    "data_an019_2013_08_21_210",
    "data_an019_2013_08_20_245",
    "data_an019_2013_08_20_275",
    # "data_an019_2013_08_21_240", # only one event
    # "data_an019_2013_08_21_270",
    "data_an019_2013_08_16_280",
    "data_an019_2013_08_16_310",
    "data_an019_2013_08_16_340",
    "data_an019_2013_08_16_370",
    "data_an019_2013_08_19_410",
    "data_an019_2013_08_19_440",
    "data_an019_2013_08_19_470",
    "data_an019_2013_08_15_480",
    "data_an019_2013_08_19_500",
    "data_an019_2013_08_15_510",
    "data_an019_2013_08_19_530",
    "data_an019_2013_08_15_570",
    "data_an019_2013_08_21_600",
    "data_an019_2013_08_21_630",
]

## Dynamical analysis

Analysis for each session.

On average, the longest responses were for the 4th trial. We take it.

In [4]:
global_events = [[] for i in range(56)] # by session
global_events_vector = [[] for i in range(56)] # by session
global_events_sec = []
global_events_duration = []
global_cluster_number = []
global_cluster_selfsimilarity = [[] for i in range(56)]

core_reproducibility_perc = 99 # threshold for detecting cores

sessions_evt_toBremoved = [[] for i in range(56)] # by session
sessions_evt_toBremoved[1] = [0] 

all_degrees = []
all_core_degrees = []
all_local_clustering_coefficients = []
all_core_local_clustering_coefficients = []
all_pagerank_cores = []
all_pagerank_others = []

for scan_id,oe in enumerate(ophys_experiments):
    # pprint.pprint(oe)

    print("\n\n%d/%d - %s" % (scan_id+1,len(ophys_experiments),oe))

    if os.path.exists(exp_path+'/spiketrains_'+str(oe)+'.npy') and os.path.exists(exp_path+'/time_'+str(oe)+'.npy'):
        scan_spiketrains = np.load(exp_path+'/spiketrains_'+str(oe)+'.npy', allow_pickle=True)
        time = np.load(exp_path+'/time_'+str(oe)+'.npy', allow_pickle=True)
        print("... loaded",len(scan_spiketrains), "spiketrains")
        # print(time)
    
    start_time = 0
    exp_tstart = start_time
    stop_time = max([max(st) if len(st) else 0 for st in scan_spiketrains])
    
    print("... producing spike rasterplot per session")
    fig = plt.figure()
    for row,train in enumerate(scan_spiketrains):
        plt.scatter( train, [row]*len(train), marker='o', edgecolors='none', s=1, c='k' )
    plt.ylabel("cell IDs")
    plt.xlabel("time (s)")
    fig.savefig(exp_path+'/results/rasterplot_scan%s.png'%scan_id, transparent=True, dpi=800)
    plt.close()
    fig.clear()
    fig.clf()
    
    ophys_cell_ids = list(range(len(scan_spiketrains)))
    ophys_cell_indexes = ophys_cell_ids # alias
    extra_toBremoved = sessions_evt_toBremoved[scan_id]

    # --------------------------------------------------------------------------
    %run "dynamical_analysis.ipynb"

    global_events[scan_id].extend(events)
    global_events_vector[scan_id].extend(events_vectors)
    global_events_sec.append(events_sec)
    global_events_duration.extend(events_durations_f)
    global_cluster_number.append(nclusters)
    global_cluster_selfsimilarity.extend(reproducibility_list)

    # --------------------------------------------------------------------------
    # Functional structure analysis
    spiketrains = scan_spiketrains
    perc_corr = 0.6
    %run "functional_analysis.ipynb"
    all_degrees.extend(degrees)
    all_core_degrees.extend(core_degrees)
    all_local_clustering_coefficients.extend(local_clustering_coefficients)
    all_core_local_clustering_coefficients.extend(core_local_clustering_coefficients)
    all_pagerank_cores.extend(pagerank_cores)
    all_pagerank_others.extend(pagerank_others)

# modularity
print('... Total modularity')
# figure
fig, (hmmap, chist) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [6, 1]})
# hierarchy
hmmap.scatter( all_degrees, all_local_clustering_coefficients, marker='o', facecolor='#111111', s=50, edgecolors='none', alpha=0.5) 
hmmap.scatter( all_core_degrees, all_core_local_clustering_coefficients, marker='o', facecolor='none', s=50, edgecolors='forestgreen') 
hmmap.set_yscale('log')
hmmap.set_ylim([0.02,1.1])
hmmap.set_xscale('log')
hmmap.spines['top'].set_visible(False)
hmmap.spines['right'].set_visible(False)
hmmap.set_ylabel('LCC')
hmmap.set_xlabel('degree')
hmmap.tick_params(axis='both', bottom='on', top='on', left='off', right='off')
# core lcc histogram
bins = np.linspace(0.02,1,50)
barheight = (max(all_local_clustering_coefficients)-min(all_local_clustering_coefficients))/50
lcc_hist, lcc_binedges = np.histogram(all_core_local_clustering_coefficients, bins)
chist.barh(bins[:-1], lcc_hist, height=barheight, align='center', color='green', linewidth=0)
chist.spines['top'].set_visible(False)
chist.spines['right'].set_visible(False)
chist.tick_params(axis='x', which='both', bottom=True, top=False, labelsize='x-small')
chist.tick_params(axis='y', which='both', left=True, right=False, labelleft=True)
chist_ticks = chist.get_xticks()
chist.set_ylim([0.01,1.1])
chist.set_ylabel('LCC')
chist.set_xlabel('count')
chist.yaxis.set_label_position("right")
chist.spines['top'].set_visible(False)
chist.spines['right'].set_visible(False)
chist.spines['bottom'].set_visible(False)
plt.tight_layout()
fig.savefig(exp_path+"/results/cores_hierarchical_modularity_all.svg", transparent=True)
plt.close()
fig.clear()
fig.clf()

# totals
print('... Total PageRank centrality')
# description
print("    cores: "+str(stats.describe(all_pagerank_cores)) )
print("    others: "+str(stats.describe(all_pagerank_others)) )
# significativity
print("    Kruskal-Wallis test:  %.3f p= %.3f" % stats.kruskal(all_pagerank_cores, all_pagerank_others))
d,_ = stats.ks_2samp(all_pagerank_cores, all_pagerank_others) # non-parametric measure of effect size [0,1]
print('    Kolmogorov-Smirnov Effect Size: %.3f' % d)

fig, ax = plt.subplots()
xs = np.random.normal(1, 0.04, len(all_pagerank_cores))
plt.scatter(xs, all_pagerank_cores, alpha=0.3, c='forestgreen')
xs = np.random.normal(2, 0.04, len(all_pagerank_others))
plt.scatter(xs, all_pagerank_others, alpha=0.3, c='silver')
vp = ax.violinplot([all_pagerank_cores,all_pagerank_others], widths=0.15, showextrema=False, showmedians=True)
for pc in vp['bodies']:
    pc.set_edgecolor('black')
for pc,cb in zip(vp['bodies'],['#228B224d','#D3D3D34d']):
    pc.set_facecolor(cb)
vp['cmedians'].set_color('orange')
vp['cmedians'].set_linewidth(2.)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.ylabel('PageRank')
plt.xticks([1, 2], ["core\n(n={:d})".format(len(all_pagerank_cores)), "other\n(n={:d})".format(len(all_pagerank_others))])
fig.savefig(exp_path+"/results/global_cores_others_pagerank_all.svg", transparent=True)
plt.show()



1/17 - data_an019_2013_08_21_210
... loaded 49 spiketrains
... producing spike rasterplot per session
    population firing: 6.11±4.16 sp/frame
    cells firing rate: 0.12±0.37 sp/s
... generating surrogates to establish population event threshold
... loaded surrogates
    event size threshold (mean): 17.44482723166876
    find peaks
    find minima
    find population events
    signatures of population events
    number of events: 7
    number of events per sec: 0.01362539349422875
    events duration: 1.200±0.512
    events size: 35.000±2.119
    Similarity of events matrix
    clustering - linkage
    surrogate events signatures for clustering threshold
   ... loaded surrogates
    cluster reproducibility threshold: 0.0
    cluster size threshold: 2
    Total number of clusters: 4
    # clusters (after removing those below reproducibility threshold): 2
    event color assignment
    unique color assignments
    clustered similarity map
    finding cluster cores
    removing cores

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.10/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


    full adjacency matrix: (49, 49)
    checking details of best cross-correlation pairs 


KeyboardInterrupt: 

KeyboardInterrupt: 